In [79]:
import pandas as pd
import os
import plotly.express as px
import plotly.io as pio
from plotly import graph_objects as go
import unicodedata
import re
import Levenshtein
pio.renderers.default="vscode+pdf"
pio.templates["custom"] = pio.templates["plotly_white"]
pio.templates["custom"]["layout"]["font"] = {"size": 15}
pio.templates.default = "custom"

# Validation de la simulation de référence

La validation de la simulation de référence repose sur les données ouvertes fournies par
Île-de-France Mobilités, correspondant aux deux derniers semestres de 2024.

Il est à noter que certaines lignes ou stations ont pu subir des modifications pendant cette
période, ainsi qu’entre le 1er janvier 2025 et le 17 février 2025, date de récupération des
données GTFS. Par exemple, la station Villejuif - Gustave Roussy, ouverte en janvier
2025, est présente dans la simulation mais absente des données de référence.

Les données utilisées sont réparties en deux catégories :
- les données des voies ferrées, correspondant aux validations à chaque arrêt (hors
correspondances),
- et les données de surface, concernant les lignes de bus et la majorité des tramways,
disponibles sous forme agrégée par ligne

Nous séparerons donc cette analyse en deux, entre le réseau ferré et le réseau de surface.

In [80]:
# Read data
fact = 100

gtfs_folder = r"..\implementation_gtfs\data\IDFM-gtfs-june"
routes = pd.read_csv(os.path.join(gtfs_folder, "routes.txt"))
stops = pd.read_csv(os.path.join(gtfs_folder, "stops.txt"))

# Simulation baseline
baseline_folder = os.path.join("baseline_output","1_percent_june")
baseline_legs = pd.read_csv(os.path.join(baseline_folder, "eqasim_legs.csv"), sep = ";")
baseline_pt = pd.read_csv(os.path.join(baseline_folder, "eqasim_pt.csv"), sep = ";")
baseline_trips = pd.read_csv(os.path.join(baseline_folder, "eqasim_trips.csv"), sep = ";")

# Reference data (2023)
annual_frequency_metro = pd.read_csv("validation_data/data_2013_wiki/freq_metro.csv")
annual_frequency_tram = pd.read_csv("validation_data/data_2013_wiki/freq_tram.csv")

# Reference data (2024)
validation_folder = "validation_data\csv"

train_files = []
surface_files = []

for file in os.listdir(validation_folder):
    if "ferre" in file:
        train_files.append(file)

    if "surface" in file:
        surface_files.append(file)

train_data = pd.DataFrame()

for file in train_files:
    temp_df = pd.read_csv(os.path.join(validation_folder, file),sep=";")
    train_data = pd.concat([train_data, temp_df])

surface_data = pd.DataFrame()

for file in surface_files:
    temp_df = pd.read_csv(os.path.join(validation_folder, file),sep=";",low_memory=False)
    surface_data = pd.concat([surface_data, temp_df])

In [81]:
pt_modes = baseline_pt["transit_mode"].unique()
count_lines_baseline = {mode: {} for mode in pt_modes if mode != "bus"}

for mode in pt_modes:
    if mode != "bus":
        tmp_control = baseline_pt[baseline_pt.transit_mode == mode]
        lines_baseline = tmp_control["transit_line_id"].unique()

        for line in lines_baseline:
            name = routes[routes.route_id == line]
            if not name.empty:
                name = name["route_short_name"].values[0]
                if name not in count_lines_baseline[mode]:
                    count_lines_baseline[mode][name] = 0
                count_lines_baseline[mode][name] += len(tmp_control[tmp_control.transit_line_id == line])*fact

In [82]:
annual_frequency_metro["ref"] = annual_frequency_metro["ref"]*1000000/365
annual_frequency_metro["line"] = annual_frequency_metro["line"].astype(str)

ref = {row["line"] : row["ref"] for _, row in annual_frequency_metro.iterrows()}

count_subway = count_lines_baseline["subway"].copy()
count_subway["7"] = count_subway["7"] + count_subway["7B"]
count_subway["3"] = count_subway["3"] + count_subway["3B"]
count_subway = {key : value for key, value in count_subway.items() if key not in ["3B", "7B"]}
count_subway = {key : value for key, value in count_subway.items() if value != 0}

annual_frequency_metro["simu"] = annual_frequency_metro["line"].map(count_subway)

annual_frequency_metro = annual_frequency_metro.sort_values(by="ref", ascending=True)
subway_2023 = px.bar(annual_frequency_metro.rename(columns={"ref" : "Données de validation", "simu" : "Simulation de référence"}), x="line", y=["Données de validation", "Simulation de référence"], barmode="group",
              labels={
                  "Value" : "Nombre de trajets",
                  "line" : "Ligne de métro"
              },
              title="Nombre de trajets avec les données de référence (2023) et la simulation baseline")
subway_2023.write_image("outputs/plots/validation/nb_trajets_2023.png", width=1000)

annual_frequency_metro["diff_abs"] = abs(annual_frequency_metro["ref"] - annual_frequency_metro["simu"])
annual_frequency_metro = annual_frequency_metro.sort_values(by="diff_abs", ascending=True)
subway_2023_diff = px.bar(annual_frequency_metro, x="line", y="diff_abs", title="Différence absolue entre ref et simu")

annual_frequency_metro["diff_percent"] = abs(((annual_frequency_metro["simu"] - annual_frequency_metro["ref"]) / annual_frequency_metro["ref"]) * 100)
annual_frequency_metro = annual_frequency_metro.sort_values(by="diff_percent", ascending=True)
subway_2023_diff_percent = px.bar(annual_frequency_metro, x="line", y="diff_percent", title="Différence en pourcentage entre ref et simu")

## Réseau ferré

Le réseau ferré est composé des lignes demandant une validation à un arrêt physique pour y accéder. Il comporte les lignes de TER, RER, Transilien, Métro ainsi que les tramways 4, 11, 12, 13.

La majorité des correspondances entre ligne ne nécessite pas de validation. Par exemple, à la Motte-Picquet Grenelle, le passage du métro 10 au métro 8 peut se faire sur le même quai. Pour cette raison, les correspondances ne sont pas inclues dans les données ouvertes de l'IDFM. Les résultats de la simulation de référence ont été traités en prenant en compte cette information.

Dans un même trajet, seuls les tronçons de type subway ou rail sont retenus, à condition qu’ils ne soient pas précédés par un autre tronçon de même type, et que les identifiants des arrêts de départ et d’arrivée correspondent.

In [83]:
def clean_string(string : str) -> str:
    """
    Cleans a string by removing accents, special characters, and converting it to lowercase.

    :param string: The original string to be cleaned.
    :return: A cleaned string with no accents, special characters, and in lowercase.
    """

    normalized_string = unicodedata.normalize('NFD', string)

    no_accent_string = ''.join(
        c for c in normalized_string if unicodedata.category(c) != "Mn"
    )

    cleaned_string = re.sub('[^0-9a-zA-Z]+', '', no_accent_string).lower()

    return cleaned_string


In [84]:
train_data["JOUR"] = pd.to_datetime(train_data["JOUR"])
train_data = train_data[~train_data["JOUR"].dt.weekday.isin([5,6])]

surface_data["JOUR"] = pd.to_datetime(surface_data["JOUR"])
surface_data = surface_data[~surface_data["JOUR"].dt.weekday.isin([5,6])]

In [85]:
nb_dates = len(train_data["JOUR"].unique())

# Données de validation
train_data["NB_VALD"] = train_data["NB_VALD"].apply(lambda x: str(x).replace(" ","")).astype(int)

anomalies = train_data[train_data.ID_ZDC.isna()]

train_data = train_data[train_data.ID_ZDC.notna()]
train_data["ID_ZDC"] = train_data["ID_ZDC"].astype(int).astype(str)

train_data = train_data.groupby(['LIBELLE_ARRET', "ID_ZDC"], as_index=False)['NB_VALD'].sum()
anomalies = anomalies.groupby('LIBELLE_ARRET', as_index=False)['NB_VALD'].sum()

stations = train_data["LIBELLE_ARRET"].unique()
for name in anomalies["LIBELLE_ARRET"].unique():
    best_stop = None
    best_ratio = 0

    for stat in stations:
        ratio = Levenshtein.ratio(clean_string(name), clean_string(stat))
        if ratio > best_ratio:
            best_ratio = ratio
            best_stop = stat

    train_data.loc[train_data.LIBELLE_ARRET == best_stop, "NB_VALD"] += anomalies.loc[anomalies.LIBELLE_ARRET == name, "NB_VALD"].values[0]

train_data["ID_ZDC"] = "IDFM:" + train_data["ID_ZDC"]

train_data = train_data.merge(
    stops[['stop_id', 'stop_name']],
    left_on='ID_ZDC',
    right_on='stop_id',
    how='left'
).rename(columns={'stop_name': 'name'}).drop('stop_id', axis=1)

anomalies = train_data[train_data.name.isna()]

parent_stops = stops[stops.location_type == 1]

for _, arret in anomalies.iterrows():
    name = arret["LIBELLE_ARRET"]

    if arret["ID_ZDC"] == "IDFM:999999":
        #print(f"Skipped {name}, with {arret['NB_VALD']} entries")
        continue

    best_match = None
    best_id = None
    best_ratio = 0

    for _, row in parent_stops.iterrows():
        station = row["stop_name"]
        ratio = Levenshtein.ratio(clean_string(name), clean_string(station))
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = station
            best_id = row["stop_id"]

    if best_ratio > 0.65:
        train_data.loc[train_data.LIBELLE_ARRET == name, "name"] = best_match
        train_data.loc[train_data.LIBELLE_ARRET == name, "ID_ZDC"] = best_id

    else:
        print(f"Did not find a good match for {name} [best_match is {best_match}, ratio : {best_ratio}]")

train_data["NB_VALD"] /= nb_dates

Did not find a good match for OZOIR LA FERRIE [best_match is Henri Mondor - Laferrière, ratio : 0.6470588235294117]


In [86]:
baseline_train = baseline_pt.sort_values(by=["person_id", "person_trip_id", "leg_index"]).reset_index(drop=True)

baseline_train["prev_egress_area_id"] = baseline_train.groupby(["person_id", "person_trip_id"])["egress_area_id"].shift(1)
baseline_train["prev_transit_mode"] = baseline_train.groupby(["person_id", "person_trip_id"])["transit_mode"].shift(1)

mask = (
    (baseline_train["access_area_id"] == baseline_train["prev_egress_area_id"]) & 
    (baseline_train["transit_mode"].isin(["rail", "subway"])) & 
    (baseline_train["prev_transit_mode"].isin(["rail", "subway"]))
)

baseline_train = baseline_train[~mask].drop(columns=["prev_egress_area_id", "prev_transit_mode"])
baseline_train = baseline_train[(baseline_train.transit_mode != "bus") & (baseline_train.transit_mode != "tram")]

baseline_train_count = baseline_train["access_area_id"].value_counts().reset_index()
baseline_train_count["count"] *= fact

baseline_train_count = baseline_train_count.merge(
    stops[['stop_id', 'stop_name']],
    left_on='access_area_id',
    right_on='stop_id',
    how='left'
).rename(columns={'stop_name': 'name'}).drop('stop_id', axis=1)


In [114]:
merge_data = train_data.merge(baseline_train_count, on = "name")
merge_data = merge_data.rename(columns={"count" : "Simulation de référence", "NB_VALD" : "Données de validation"})
melted_data = merge_data.melt(id_vars='name', value_vars=['Données de validation', 'Simulation de référence'], var_name='Metric', value_name='Value')
merge_data_sample = merge_data.sample(10)
melted_data_sample = merge_data_sample.melt(id_vars='name', value_vars=['Données de validation', 'Simulation de référence'], var_name='Metric', value_name='Value')

fig = px.histogram(melted_data, x='name', y='Value', color='Metric', barmode='group',
                   labels={"name": "Arrêt", "Value" : "Nombre de validations"}, title='Comparaison entre la référence et la simulation')
fig.write_image("outputs/plots/validation/comp_ref_simu_ferre.png", width=1000)

merge_data['Percentage_Difference'] = (
    (merge_data['Simulation de référence'] - merge_data['Données de validation']) / merge_data['Données de validation']
) * 100

filtered_data = merge_data[merge_data['name'].isin(merge_data['name'].to_list())]
names = ["Gare Montparnasse", "Gare de Lyon", "Gare de l'Est", "Aéroport CDG - Terminal 2 (TGV)", "Gare Saint-Lazare"]
merge_data = merge_data[merge_data["name"].isin(names)]
melted_data = merge_data.melt(id_vars='name', value_vars=['Données de validation', 'Simulation de référence'], var_name='Metric', value_name='Value')

fig = px.histogram(melted_data, x='name', y='Value', color='Metric', barmode='group',
                   labels={"name": "Arrêt", "Value" : "Nombre de validations"}, title='Comparaison entre la référence et la simulation')

fig_diff = px.bar(filtered_data, x='name', y='Percentage_Difference',
                  title='Différence (en %) entre les données de référence et la simulation baseline pour chaque arrêt',
                  labels={'Percentage_Difference': 'Différence (%)', "name": "Arrêt"})

fig_diff.write_image("outputs/plots/validation/comp_ref_simu_ferre_pourcent.png", width=1000)
sum_data = melted_data.groupby("Metric")["Value"].sum().reset_index()

reference_value = sum_data.loc[sum_data["Metric"] == "Données de validation", "Value"].values[0]
simulation_value = sum_data.loc[sum_data["Metric"] == "Simulation de référence", "Value"].values[0]
diff_percentage = ((simulation_value - reference_value) / reference_value) * 100

fig2 = px.bar(sum_data, x="Metric", y="Value", title="Somme totale des valeurs")

# Ajouter une annotation pour afficher la différence en pourcentage
fig2.add_annotation(
    x=0.5, y=max(reference_value, simulation_value),  # Position
    text=f"Différence : {diff_percentage:.2f}%",  # Texte formaté
    showarrow=False,
    font=dict(size=14, color="red"),
    xref="paper", yref="y"
)

fig2.write_image("outputs/plots/validation/comp_ref_simu_ferre_total.png", width=1000)

fig_scatter = px.scatter(
    merge_data, 
    x="Données de validation", 
    y="Simulation de référence",
    hover_data=["name"], 
    trendline="ols",
    title = "Écarts entre les données de validation et la simulation par arrêt (trajets quotidiens)")
fig_scatter.data[0].name = "Arrêts"
fig_scatter.data[0].showlegend = True
fig_scatter.data[1].line.color = 'black'
fig_scatter.data[1].line.width = 2
fig_scatter.data[1].name = "Régression linéaire"
fig_scatter.data[1].showlegend = True
fig_scatter.add_trace(go.Scatter(x=[0,merge_data["Données de validation"].max()], y=[0,merge_data["Données de validation"].max()],mode="lines", name="Idéal (x = y)",line=dict(color="red", dash="dash")))

fig_scatter.write_image("outputs/plots/validation/comp_ref_simu_ferre_scatter.png", width=1000)

In [115]:
fig_scatter.show()

Sur la figure ci-dessus, est représenté la correspondance entre la simulation et les données de validation du nombre de trajets quotidiens pour chaque arrêt du réseau ferré. 

Cela montre que la simulation tend à surestimer le nombre de validations pour les arrêts faiblement fréquentés (moins de 15 000 validations), puis à sous-estimer progressivement les volumes plus elevées.

Cette sous-estimation, notamment présente dans les grandes gares (comme Saint-Lazare), peut s'expliquer par le fait que seule la population résidant en Île-de-France est prise en compte dans la simulation. Les voyageurs longue distance (TGV), les touristes ou les usagers des
aéroports ne sont donc pas représentés, ce qui conduit logiquement à une sous-estimation sur ces points d’intérêt majeurs.

In [116]:
fig.show()

La figure ci-dessus nous permet d'avoir un aperçu plus précis du nombre de validations pour un sample de 10 gares.

In [103]:
fig_diff.show()

Pour ces mêmes arrêts, on peut avoir un aperçu de la différence entre les données de validation et de la simulation de référence en pourcentage.

In [70]:
fig2.show()

Pour la figure suivante, on aggrège les données pour avoir un aperçu du nombre de validation total sur le réseau ferré. Ici on voit que, la simulation a tendance à surestimer les données réelles, à hauteur de 25%

## Réseau de surface

Les données sur le réseau de surface contiennent donc les lignes de bus ainsi que la majorité des lignes de tramway. 

L'identifiant utilisé pour une ligne dans la base de données de velidation n'a pas de correspondance directe avec les données GTFS fournies par l'Île-de-France Mobilités. Pour avoir une correspondance, on récupère le référentiel de lignes dans les données ouvertes de l'IDFM.

Une fois la correspondance faite, on peut désormais analyser les résultats.

In [71]:
surface_baseline = baseline_pt[(baseline_pt.transit_mode == "bus") | (baseline_pt.transit_mode == "tram")]

nb_dates = len(surface_data["JOUR"].unique())
referentiel_lines = pd.read_csv(os.path.join("validation_data", "referentiel-des-lignes.csv"), sep=";")
selected_columns = ['ID_Line', 'ID_GroupOfLines']
referentiel_lines = referentiel_lines[selected_columns]

surface_data = surface_data.groupby("ID_GROUPOFLINES", as_index=False)["NB_VALD"].sum()

surface_data = surface_data.merge(
    referentiel_lines[['ID_GroupOfLines', 'ID_Line']],
    left_on = "ID_GROUPOFLINES",
    right_on = "ID_GroupOfLines",
    how = "left"
)

surface_data["ID_Line"] = "IDFM:" + surface_data["ID_Line"]

surface_data = surface_data.merge(
    routes[["route_id", "route_short_name", "route_type"]],
    left_on = 'ID_Line',
    right_on = "route_id",
    how = "left"
)

bus_tram = ["BUS T2", "T1B"]
surface_data = surface_data[~surface_data["route_short_name"].isin(bus_tram)]

surface_data = surface_data.drop(columns=["ID_GroupOfLines", "ID_Line"])

surface_baseline = surface_baseline["transit_line_id"].value_counts().reset_index()
surface_baseline["count"] *= fact

surface_data = surface_data.merge(
    surface_baseline[["transit_line_id", "count"]],
    left_on = "route_id",
    right_on = "transit_line_id",
    how="left"
)

anomalies = surface_data[surface_data.route_id.isna()]
surface_data = surface_data[surface_data.route_id.notna()]
surface_data["route_type"] = surface_data["route_type"].astype(int)
surface_data = surface_data[(surface_data.route_type == 0) | (surface_data.route_type == 3)]
surface_data["count"] = surface_data["count"].fillna(0)
surface_data["NB_VALD"] /= nb_dates

surface_data = surface_data.rename(columns = {"NB_VALD" : "Données de validation", "count" : "Simulation de référence"})

surface_data["route_type"]=surface_data["route_type"].astype(str)
fig_scatter_surface = px.scatter(surface_data, x ="Données de validation", y = "Simulation de référence", hover_data="route_short_name", color="route_type", trendline="ols")
fig_scatter_surface.data[0].name = "Lignes de bus"
fig_scatter_surface.data[0].showlegend = True
fig_scatter_surface.data[1].line.color = 'cyan'
fig_scatter_surface.data[1].line.width = 2
fig_scatter_surface.data[1].name = "Régression linéaire (bus)"
fig_scatter_surface.data[1].showlegend = True
fig_scatter_surface.data[2].name = "Lignes de tram"
fig_scatter_surface.data[2].showlegend = True
fig_scatter_surface.data[3].line.color = 'orange'
fig_scatter_surface.data[3].line.width = 2
fig_scatter_surface.data[3].name = "Régression linéaire (tramway)"
fig_scatter_surface.data[3].showlegend = True
fig_scatter_surface.add_trace(go.Scatter(x=[0,surface_data["Données de validation"].max()], y=[0,surface_data["Données de validation"].max()],mode="lines", name="Idéal (x = y)",line=dict(color="red", dash="dash")))

fig_scatter_surface.write_image("outputs/plots/validation/comp_ref_simu_surface_scatter.png", width="1000")

surface_data["route_type"]=surface_data["route_type"].astype(int)

surface_data = surface_data[(surface_data.route_type == 0)]

surface_data = surface_data.groupby(["ID_GROUPOFLINES", "Données de validation", "route_short_name"], as_index="False")["Simulation de référence"].sum().reset_index()

melted_data = surface_data.melt(id_vars='route_short_name', value_vars=['Données de validation', 'Simulation de référence'], var_name='Metric', value_name='Value')

In [72]:
fig_scatter_surface.show()

La figure ci-dessus montre la correspondance du nombre de validations pour un jour, pour chaque ligne du réseau de surface. On remarque que les bus correspondent relativement bien à l'idéal. Quant aux tramways, ceux-ci sont surestimés dans la simulation. 

In [73]:
fig_surface_comp = px.histogram(melted_data, 
                   x='route_short_name', 
                   y='Value', 
                   color='Metric', 
                   barmode='group',
                   labels={
                       "route_short_name" : "Ligne de Tramway",
                       "Value" : "Nombre de trajets"
                   },
                   title='Comparaison entre les données de référence et la simulation baseline')

fig_surface_comp.write_image("outputs/plots/validation/comp_ref_simu_surface.png", width=1000)

surface_data['Différence (%)'] = ((surface_data['Simulation de référence'] - surface_data['Données de validation']) / surface_data['Données de validation']) * 100
diff_data = surface_data[['route_short_name', 'Différence (%)']]

fig_surface_diff = px.bar(diff_data, 
                  x='route_short_name', 
                  y='Différence (%)', 
                  labels={
                      "route_short_name" : "Ligne de Tramway"
                  },
                  title='Différence (en %) entre les données de référence et la simulation baseline')

fig_surface_diff.write_image("outputs/plots/validation/comp_ref_simu_surface_pourcentage.png", width=1000)

In [74]:
fig_surface_comp.show()

Ici on a une vision plus précise du nombre de trajets pour les lignes de tramways.

In [75]:
fig_surface_diff.show()

In [ ]:
#!jupyter nbconvert --to pdf --no-input validation_baseline.ipynb

[NbConvertApp] Converting notebook validation_baseline.ipynb to pdf
[NbConvertApp] Support files will be in validation_baseline_files\
[NbConvertApp] Making directory .\validation_baseline_files
[NbConvertApp] Writing 26413 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | b had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 174325 bytes to validation_baseline.pdf
